In [1]:
import json
import numpy as np
import pandas as pd
import lxml
import re
import os
from datetime import datetime
import pyodbc
import requests

In [12]:
custom_headers  = {"Authorization": "Gt4qkl1ahFDcFyQwfjMBYT0utbLD4STHQjWdIXzsYH8v3w29M6Tm6tVKI0uMj6Qh", "Content-Type": "application/json"}
url = 'https://ea.executive.automats.app/api/bi/scripts?withHistory=true'

resp = requests.get(url, headers = custom_headers, verify=False)
print(resp.status_code)

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:999: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ea.executive.automats.app'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


200


In [13]:
json_data = json.loads(resp.text)
json_text = json.dumps(json_data, indent=4)

In [14]:
try:
    os.remove('data/execution/try.json')
except:
    print('File does not exist')

with open('data/execution/try.json', 'w') as fw:
    fw.write(json_text)

In [15]:
try:
    os.remove('data/execution/script_history_fixed.json')
except:
    print('File does not exist')

#with open('data/execution/script_history.json', 'r') as fr:
with open('data/execution/try.json', 'r') as fr:    
    # reading line by line
    lines = fr.readlines()
    last_line = len(lines)
    print(len(lines))
    
    for line in lines:
        line = re.sub("\u0003", "", line)
        with open('data/execution/script_history_fixed.json', 'a') as fw:
            fw.write(line) 

652355


In [2]:
f = open('data/execution/script_history_fixed.json')
data = json.loads(f.read())

In [3]:
try:
    os.remove('data/execution/check_structure.csv')
except:
    print('File does not exist')
with open('data/execution/check_structure.csv', 'a') as fw:
    line = 'records_no,topProjectName,name,sessionId,id,scriptId,scriptName,state,status,label,startTime,endTime,duration,lineNum,StepLogs_id\n'
    fw.write(line)
    i = 0
    for level1 in data:
        for key, value in level1.items():
            #if i == 100: break
            i = i + 1
            try:
                executionLogs_str = json.dumps(level1['executionLogs'])
                executionLogs_json = json.loads(executionLogs_str)
                for level2 in executionLogs_json:
                    for key2, value2 in level2.items():
                        #try:
                        executionStepLogs_str = json.dumps(level2['executionStepLogs'])
                        executionStepLogs_json = json.loads(executionStepLogs_str)
                        if key2 == 'executionStepLogs':
                            records_no_str = str(i)
                            topProjectName = str(level1['topProjectName'])
                            name_str = str(level1['name'])
                            # Remove any commas from name 
                            check = (',' in name_str)
                            if check == True:
                                name_str = re.sub(',', ';', name_str)                            
                            sessionId_str = str(level2['sessionId'])
                            id_str = str(level2['id'])
                            scriptId_str = str(level2['scriptId'])
                            scriptName_str = str(level2['scriptName'])
                            # Remove any commas from scriptName 
                            check = (',' in scriptName_str)
                            if check == True:
                                scriptName_str = re.sub(',', ';', scriptName_str)                             
                            state_str = str(level2['state'])
                            status_str = str(level2['status'])
                            lastIssues_str = str(level2['lastIssues'])
                            # Remove any commas from lastIssues_str
                            check = (',' in lastIssues_str)
                            if check == True:
                                lastIssues_str = re.sub(',', ';', lastIssues_str)


                            for level3 in executionStepLogs_json:
                                #for key3, value3 in level3.items():
                                status_str = str(level3['status'])
                                label_str = str(level3['label'])
                                
                                # These lines replace long text test step labels with short text (that does not wrap)
                                check = ('Click Dynamics 365' in label_str)
                                if check == True:
                                    label_str = 'Click Dynamics 365'
                                check = ('Click Sales Hub' in label_str)
                                if check == True:
                                    label_str = 'Click Sales Hub'
                                check = ('Type value function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Type value function eaExecuteVariableValue()'
                                check = ('Select from function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select from function eaExecuteVariableValue()'                                    
                                check = ('Select lookup values function eaExecuteVariableValue()' in label_str)
                                if check == True:
                                    label_str = 'Select lookup values function eaExecuteVariableValue()'
                                # Remove any commas from label_str 
                                check = (',' in label_str)
                                if check == True:
                                    label_str = re.sub(',', ';', label_str)
                                # Now replace all newlines that remain in label_str
                                check = ('\n' in label_str)
                                if check == True:
                                    label_str = re.sub('\n', '/nl/', label_str)
                                # Done with replacing text

                                startTime_str = str(level3['startTime'])
                                endTime_str = str(level3['endTime'])
                                duration_str = str(level3['duration'])
                                lineNum_str = str(level3['lineNum'])
                                StepLogs_id_str = str(level3['id'])

                                line = records_no_str+','+topProjectName+','+name_str+','+sessionId_str+','+id_str+','+scriptId_str+','+scriptName_str+','+state_str+','+status_str+','+label_str+','+startTime_str+','+endTime_str+','+duration_str+','+lineNum_str+','+StepLogs_id_str+'\n'
                                fw.write(line)
                            # except:
                            #     print('Exception on Level2 for Level1 item '+str(i))
            except:
                #print('Exception for Level1 item '+str(i))
                pass
    print('Processed '+str(i)+' of 7899 items')                


Processed 7934 of 7899 items


In [4]:
df = pd.read_csv('data/execution/check_structure.csv')

In [5]:
#df.drop_duplicates(keep='first',inplace=True)
df.rename(columns={'label':'StepLabel'}, inplace=True)
df.rename(columns={'name':'testName'}, inplace=True)

In [6]:
mask = (df['startTime'] == 'None') 
df['startTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['startTime'] = pd.to_datetime(df['startTime'])

In [7]:
mask = (df['endTime'] == 'None') 
df['endTime'].mask(mask,'2022-01-01T00:00:00.000Z',inplace=True)
df['endTime'] = pd.to_datetime(df['endTime'])

In [8]:
df['duration_ms'] = df['endTime'] - df['startTime']

In [9]:
mask = (df['duration'] == 'None') 
df['duration'].mask(mask,'0',inplace=True)
df['duration'] = pd.to_numeric(df['duration'])

In [ ]:
df.info()

In [10]:
df['date'] = pd.to_datetime(df['endTime'], format='%Y-%m-%d').dt.date
df['date'] = pd.to_datetime(df['date'])

In [11]:
mask = (df['date'] > '2023-01-10')
#df.loc[(df['date'] > '2023-01-10')]
df = df[mask]

In [ ]:
# https://learn.microsoft.com/en-us/sql/machine-learning/data-exploration/python-dataframe-pandas?view=sql-server-ver16
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = 'tcp:csazniptlsql01.database.windows.net,1433' 
database = 'CSAZN-INT-IPT-L-DSQ-01' 
username = 'ccbaadmin' 
password = '@FireBase123' 
# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
cursor = cnxn.cursor()

cursor.execute('''
		CREATE TABLE ea_script_history (
			records_no int,
            topProjectName nvarchar(255),
            testName nvarchar(255),
            sessionId nvarchar(255),
            id int,
            scriptId int,
            scriptName nvarchar(255),
            state nvarchar(255),
            status nvarchar(255),
            StepLabel nvarchar(255),
            startTime datetime,
            endTime datetime,
            duration int,
            lineNum int,
            StepLogs_id int
			)
               ''')

cnxn.commit()

In [ ]:
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
i = 0
for index, row in df.iterrows():
     cursor.execute("INSERT INTO ea_script_history (records_no,topProjectName,testName,sessionId,id,scriptId,scriptName,state,status,StepLabel,startTime,endTime,duration,lineNum,StepLogs_id) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row.records_no, row.topProjectName, row.testName, row.sessionId, row.id, row.scriptId, row.scriptName, row.state, row.status, row.StepLabel, row.startTime, row.endTime, row.duration, row.lineNum, row.StepLogs_id)
     i = i + 1
     if i == 1000:
          i = 0
          cnxn.commit()          
cnxn.commit()
cursor.close()

In [ ]:
# cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=yes;UID='+username+';PWD='+ password)
# cursor = cnxn.cursor()
# # Insert Dataframe into SQL Server:
# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO ea_script_history (startTime) values(?)", row.startTime)
# cnxn.commit()
# cursor.close()

In [ ]:
# Delete all the rows in the SQl script history table
cursor = cnxn.cursor()
cursor.execute("DELETE FROM ea_script_history")
cnxn.commit()
cursor.close()

In [ ]:
# Drop the SQl script history table (it must be re-created)
cursor = cnxn.cursor()
cursor.execute("DROP Table ea_script_history")
#cursor.execute("DROP Table temp_table")
cnxn.commit()
cursor.close()

In [ ]:
line = '},        "tags": [],        "useDefaultEnvironment": true, }abc        "tags": []abc        "useDefaultEnvironment": true,'

line = re.sub(r"\},[^,]", "},\n", line) # OK
line = re.sub(r"\],[^,]", "],\n", line) # OK
line = re.sub(r"\}[^,]", "}\n", line) # OK
line = re.sub(r"\][^,]", "]\n", line) # OK


In [ ]:
print(line)